# Extract from repeating and non-repeating groups into multiple output streams

Sometimes you have no choice but extract from three different forms at the same time to get 
the same info. This somewhat more complicated example does that with data structures that are extensible, somewhat

This script does two things: it looks for all the places salaries for employees or officers / directors might appear, and captures them into employees_detailed2017.csv. It also looks for the diversion checkbox, which organizations are supposed to check if they have 'become aware during the year of a significant diversion of the organization’s assets'--typically because they've been ripped off--into diversions2017.csv.

The example illustrates how to capture repeating groups and non-repeating entries from form parts. This use the StreamExtractor class to keep everything straight.


In [152]:
output_streams = {
    'employees': {  
        'filename':'employees_detailed', # will output to employees_detailedYYYY.csv where year is specified below
        'headers':["object_id", "name", "business_name1", "title", "org_comp", "related_comp", "other_cmp", "form", "source","ein"]
    },
    'salaries': {  
        'filename':'filer_comp', # will output to filer_compYYYY.csv where year is specified below
        'headers':["year", "ein", "object_id", "form", "source", "compensation", "income", "revenue", "assets", "expenses"]
    },
    'states': {
        'filename':'filer_states',
        'headers':["ein","state"]
    },
    'expenses': {
        'filename':'filer_expenses',
        'headers':["ein","object_id","prog_exp"]
    },
    'assets': {
        'filename':'filer_assets',
        'headers':["ein","object_id","assets_pf"]
    }


}


In [153]:
# The format we're using is this
# The stream_key used must be defined in the output stream above.

data_capture_dict = {
    'ReturnHeader990x': {
        'parts': {
            'returnheader990x_part_i': {
                'stream_key': 'states',  # 'stream_key' specifies where the output goes--must exist as a key in output_streams
                'ein': {'header':'ein'},
                'USAddrss_SttAbbrvtnCd':{'header':'state'},
            }

        },
    },
    'IRS990': {
        'parts': {
            'part_i': {
                'stream_key': 'salaries',  # 'stream_key' specifies where the output goes--must exist as a key in output_streams
                'ein': {'header':'ein'},
                'object_id': {'header':'object_id'},
                'CYSlrsCmpEmpBnftPdAmt':{'header':'compensation','default':0},
                'CYRvnsLssExpnssAmt':{'header':'income','default':0},
                'CYTtlRvnAmt':{'header':'revenue','default':0},
                'TtlAsstsEOYAmt':{'header':'assets','default':0},
                'CYTtlExpnssAmt':{'header':'expenses','default':0},
            },
            'part_ix': {
                'stream_key': 'expenses',  # 'stream_key' specifies where the output goes--must exist as a key in output_streams
                'ein': {'header':'ein'},
                'object_id': {'header':'object_id'},
                'TtlFnctnlExpnss_TtlAmt': {'header':'func_exp','default':0},
                'TtlFnctnlExpnss_PrgrmSrvcsAmt': {'header':'prog_exp','default':0}
                
            }

        },
        ## The remaining logic is for capturing salaries wherever they appear in 
        ## the 990, 990PF and 990EZ
        'groups': {
             'Frm990PrtVIISctnA': {
                'stream_key': 'employees',  # 'stream_key' specifies where the output goes--must exist as a key in output_streams
                'ein': {'header':'ein'},
                'object_id': {'header':'object_id'},
                'PrsnNm': {'header':'name'},
                'BsnssNmLn1Txt':{'header':'business_name1'},
                'TtlTxt': {'header':'title'},
                'RprtblCmpFrmOrgAmt': {
                    'header':'org_comp',
                    'default':0  # set numeric if missing
                },
                'RprtblCmpFrmRltdOrgAmt': {
                    'header':'related_comp',
                    'default':0
                },
                'OthrCmpnstnAmt':{
                    'header':'other_cmp',
                    'default':0
                }
            }
        }
    },
    'IRS990EZ': {
        'parts': {
            'ez_part_i': {
                'stream_key': 'salaries',  # 'stream_key' specifies where the output goes--must exist as a key in output_streams
                'ein': {'header':'ein'},
                'object_id': {'header':'object_id'},
                'SlrsOthrCmpEmplBnftAmt':{'header':'compensation','default':0},
                'ExcssOrDfctFrYrAmt':{'header':'income','default':0},
                'TtlRvnAmt':{'header':'revenue','default':0},
                'NtAsstsOrFndBlncsEOYAmt':{'header':'assets','default':0},
                'TtlExpnssAmt':{'header':'expenses','default':0}
            }
        },
        'groups': {
            'EZOffcrDrctrTrstEmpl': {
                'stream_key': 'employees',
                'ein': {'header':'ein'},
                'object_id': {'header':'object_id'},
                'PrsnNm': {'header':'name'},
                'BsnssNmLn1': {'header':'business_name1'},


                'TtlTxt': {'header':'title'},
                'CmpnstnAmt': {
                    'header':'org_comp',
                    'default':0
                },
                'related_comp': {'header':'related_comp','default':0},
                'composite': {  # other compensation includes benefits and other allowances for EZ, PF filers
                    'other_cmp': {
                        'EmplyBnftPrgrmAmt': {
                            'default':0
                        },
                        'ExpnsAccntOthrAllwncAmt': {
                            'default':0
                        }
                    }
                }
            },
            'EZCmpnstnHghstPdEmpl': {
                'stream_key': 'employees',
                'ein': {'header':'ein'},
                'object_id': {'header':'object_id'},
                'PrsnNm': {'header':'name'},
                'TtlTxt': {'header':'title'},
                'CmpnstnAmt': {
                    'header':'org_comp',
                    'default':0
                },
                'related_comp': {'header':'related_comp','default':0},
                'composite': {
                    'other_cmp': {
                        'EmplyBnftsAmt': {
                            'default':0
                        },
                        'ExpnsAccntAmt': {
                            'default':0
                        }
                    }
                }
            }
        }
    },
    'IRS990PF': {
        'parts': {
            'pf_part_i': {
                'stream_key': 'salaries',  # 'stream_key' specifies where the output goes--must exist as a key in output_streams
                'ein': {'header':'ein'},
                'object_id': {'header':'object_id'},
                'CmpOfcrDrTrstRvAndExpnssAmt':{'header':'compensation','default':0},
                'income': {'header':'income','default':0},
                'TtlRvAndExpnssAmt':{'header':'revenue','default':0},
                'TtlExpnssRvAndExpnssAmt':{'header':'expenses','default':0}
            },
            'pf_part_ii': {
                'stream_key': 'assets',  # 'stream_key' specifies where the output goes--must exist as a key in output_streams
                'ein': {'header':'ein'},
                'object_id': {'header':'object_id'},
                'TtlAsstsEOYFMVAmt':{'header':'assets_pf','default':0},
            }
        },
        'groups': {
            'PFOffcrDrTrstKyEmpl': {
                'stream_key': 'employees',

                'ein': {'header':'ein'},
                'object_id': {'header':'object_id'},
                'OffcrDrTrstKyEmpl_PrsnNm': {'header':'name'},
                'OffcrDrTrstKyEmpl_BsnssNmLn1': {'header':'business_name1'},
                'OffcrDrTrstKyEmpl_TtlTxt': {'header':'title'},
                'OffcrDrTrstKyEmpl_CmpnstnAmt': {
                    'header':'org_comp',
                    'default':0  # set numeric if missing
                },
                'composite': {
                    'other_cmp': {
                        'OffcrDrTrstKyEmpl_EmplyBnftPrgrmAmt': {
                            'default':0
                        },
                        'OffcrDrTrstKyEmpl_ExpnsAccntOthrAllwncAmt': {
                            'default':0
                        }
                    }
                }
            },
            'PFCmpnstnHghstPdEmpl': {
                'stream_key': 'employees',

                'ein': {'header':'ein'},
                'object_id': {'header':'object_id'},
                'CmpnstnHghstPdEmpl_PrsnNm': {'header':'name'},
                'CmpnstnHghstPdEmpl_TtlTxt': {'header':'title'},
                'CmpnstnHghstPdEmpl_CmpnstnAmt': {
                    'header':'org_comp',
                    'default':0  # set numeric if missing
                },
                'composite': {
                    'other_cmp': {
                        'CmpnstnHghstPdEmpl_EmplyBnftsAmt': {
                            'default':0
                        },
                        'CmpnstnHghstPdEmpl_ExpnsAccntAmt': {
                            'default':0
                        }
                    }
                }
            }
        }
    }
}



In [154]:
from stream_extractor import StreamExtractor
import unicodecsv as csv

In [155]:
YEAR = 2018  # THIS MUST AGREE WITH OUR OTHER DATA
extractor = StreamExtractor(output_streams, data_capture_dict, YEAR)

Initializing output stream employees_detailed2018.csv
Initializing output stream filer_comp2018.csv
Initializing output stream filer_states2018.csv
Initializing output stream filer_expenses2018.csv
Initializing output stream filer_assets2018.csv


In [156]:
# read the whole file of orgs with efilings from part 1 here, it's not very long
file_rows = [] 
# We're using the output of part 1
with open('orefilers.csv', 'rb') as infile:
    reader = csv.DictReader(infile)
    for row in reader:
        file_rows.append(row)

In [157]:
for filing_count, row in enumerate(file_rows):
    this_object_id = row['OBJECT_ID']
    tax_period = row['TAX_PERIOD_y']
    # don't need to pass taxpayer name in, but makes the output more readable
#     extractor.run_filing(this_object_id, taxpayer_name=row['TAXPAYER_NAME'])
    extractor.run_filing(this_object_id)
    filing_count += 1
    if filing_count % 100 == 0:
        print("Processed %s filings" % filing_count)
    
print("end")

Processed 100 filings
Processed 200 filings
Processed 300 filings
Processed 400 filings
Processed 500 filings
Processed 600 filings
Processed 700 filings
Processed 800 filings
Processed 900 filings
Processed 1000 filings
Processed 1100 filings
Processed 1200 filings
Processed 1300 filings
Processed 1400 filings
Processed 1500 filings
Processed 1600 filings
Processed 1700 filings
Processed 1800 filings
Processed 1900 filings
Processed 2000 filings
Processed 2100 filings
Processed 2200 filings
Processed 2300 filings
Processed 2400 filings
Processed 2500 filings
Processed 2600 filings
Processed 2700 filings
Processed 2800 filings
Processed 2900 filings
Processed 3000 filings
Processed 3100 filings
Processed 3200 filings
Processed 3300 filings
Processed 3400 filings
Processed 3500 filings
Processed 3600 filings
Processed 3700 filings
Processed 3800 filings
Processed 3900 filings
Processed 4000 filings
Processed 4100 filings
Processed 4200 filings
Processed 4300 filings
Processed 4400 filin

The output should be in employees_detailed2017.csv and diversions2017.csv

In [158]:
import pandas as pd
data = pd.read_csv("filer_comp2018.csv")
data.head()

,year,ein,object_id,form,source,compensation,income,revenue,assets,expenses
0,2018,455093195,201802839349301235,IRS990,part_i,0,-2579069,1187007977,9.713577e+07,1189587046
1,2018,930223960,201902269349301505,IRS990,part_i,409070526,92305846,816045270,1.265166e+09,723739424
2,2018,930933975,201803099349302465,IRS990,part_i,76035386,-40584478,861208485,4.175969e+08,901792963
3,2018,930386823,201930469349302328,IRS990,part_i,574883428,-40532975,934290343,6.051048e+08,974823318
4,2018,930602940,201803209349301685,IRS990,part_i,418451096,41444500,809085838,1.066148e+09,767641338


In [159]:
# drop older object_ids from filer_comp
dfsorted = data.sort_values('object_id', ascending=False).drop_duplicates(subset=['ein'])
dfsorted = dfsorted.drop(['year','form','source'],axis=1)
dfsorted.head()

,ein,object_id,compensation,income,revenue,assets,expenses
16,936021475,201943049349301554,0,8464131,195211222,100839235.0,186747091
2618,611431341,201943049349301504,112817,28175,349822,246112.0,321647
5608,471158839,201943049349201504,0,-7081,46717,11342.0,53798
6478,930801570,201943049349201074,0,-415,130,-793785.0,545
4028,931028728,201943049349200424,0,-5507,70181,55596.0,75688


In [160]:
#join orefilers data to org compensation file
tax = pd.read_csv("orefilers.csv",usecols=[6,7,9,12,13,14,15,16,20,21])
ntee = pd.read_csv("ntee.csv",usecols=[0,1])
joined = dfsorted.join(tax.set_index('OBJECT_ID'), on='object_id')
joined['ntee'] = joined['NTEE_CD'].str[:3]
#join NTEE category and drop ntee code
joined = joined.join(ntee.set_index('NTEE'), on='ntee')
joined = joined.drop(['NTEE_CD','ntee'],axis=1)
joined = joined.rename(columns={"TAX_PERIOD_y": "tax_period", "Code": "ntee"})
joined.set_index('ein', inplace=True)
values = {'compensation': 0,'income': 0,'assets': 0,'revenue': 0,'expenses':0,'tax_period': 0}
joined.fillna(value=values,inplace=True)
joined = joined.astype({'compensation': 'int64','income': 'int64','revenue': 'int64','assets': 'int64','expenses': 'int64','tax_period': 'int64'})
joined.head()
#join program expenses
exp = pd.read_csv("filer_expenses2018.csv",usecols=[1,2])
withexp = joined.join(exp.set_index('object_id'), on='object_id')
#join foundation assets
assets = pd.read_csv("filer_assets2018.csv",usecols=[1,2])
withassets = withexp.join(assets.set_index('object_id'), on='object_id')
ass_zero = {'assets_pf': 0,'prog_exp':0}
withassets.fillna(value=ass_zero,inplace=True)
withassets['assets'] = withassets['assets']+withassets['assets_pf']
withassets = withassets.astype({'assets': 'int64','prog_exp':'int64'})
withassets.drop(['assets_pf'],axis=1,inplace=True)
withassets.head()

,object_id,compensation,income,revenue,assets,expenses,TAXPAYER_NAME,RETURN_TYPE,STREET,CITY,STATE,ZIP,SUBSECTION,tax_period,ntee,prog_exp
ein,,,,,,,,,,,,,,,,
936021475,201943049349301554,0,8464131,195211222,100839235,186747091,OREGON TEAMSTER EMPLOYERS TRUST,990O,PO BOX 4148,PORTLAND,OR,97208-4148,9,201812,NaN,0
611431341,201943049349301504,112817,28175,349822,246112,321647,TUALATIN HILLS WATER POLO CLUB INC,990,16055 SW WALKER ROAD 156,BEAVERTON,OR,97006-4942,3,201812,Amateur Sports,275485
471158839,201943049349201504,0,-7081,46717,11342,53798,FRIENDS OF THE MENTOR PROGRAM,990EZ,357 N L ST,LAKEVIEW,OR,97630-1232,3,201906,Youth Development Programs,0
930801570,201943049349201074,0,-415,130,-793785,545,PALLIATIVE CARE SERVICES INC,990EZ,1620 THOMPSON RD,COOS BAY,OR,97420-2150,3,201906,"Substance Abuse Dependency, Prevention & Treat...",0
931028728,201943049349200424,0,-5507,70181,55596,75688,NEW FRIENDS OF THE BEAVERTON CITY LIBRARY,990EZ,12470 SW 5TH ST,BEAVERTON,OR,97005-0515,3,201812,Unknown,0


In [161]:
#get ids of non-oregon filers
states = pd.read_csv("filer_states2018.csv")
states.set_index('ein', inplace=True)
indexNames = states[ states['state'] != "OR" ].index
# Delete non-oregon filers
justore = withassets.drop(indexNames)
justore = justore[['object_id','TAXPAYER_NAME','STREET','CITY','STATE','ZIP','compensation','income','revenue','assets','expenses','prog_exp','tax_period','RETURN_TYPE','SUBSECTION','ntee']]
justore.head()
justore.to_csv("np_org_comp_2018.csv")


In [162]:
#read employees and drop dupes
data = pd.read_csv("employees_detailed2018.csv")
data = data.drop_duplicates()
data['org_id'] = data['ein']
ore_emp = data.set_index('ein').drop(indexNames)
ore_emp.reset_index(drop=True,inplace=True)
ore_emp.head()

,object_id,name,business_name1,title,org_comp,related_comp,other_cmp,form,source,org_id
0,201802839349301235,DALILA SARABIA,NaN,DIRECTOR,0,0.0,0,IRS990,Frm990PrtVIISctnA,455093195
1,201802839349301235,DORANE BROWER,NaN,DIRECTOR,0,0.0,0,IRS990,Frm990PrtVIISctnA,455093195
2,201802839349301235,ED BLACKBURN,NaN,DIRECTOR AND VICE CHAIR,0,0.0,0,IRS990,Frm990PrtVIISctnA,455093195
3,201802839349301235,ERIC HUNTER,NaN,"BOARD CHAIR, DIRECTOR",0,0.0,0,IRS990,Frm990PrtVIISctnA,455093195
4,201802839349301235,GEORGE J BROWN MD,NaN,"BOARD CHAIR, DIRECTOR",0,0.0,0,IRS990,Frm990PrtVIISctnA,455093195


In [163]:
#strip out employees who received no income from local org
local_emp = ore_emp[ore_emp.org_comp>0]
#make NaN's blank and concatenate name and business_name1 fields
values = {'related_comp': 0}
local_emp = local_emp.fillna(value=values)
local_emp = local_emp.astype({'related_comp': 'int64'})
local_emp = local_emp.fillna('')
local_emp.insert(3, 'emp_name', local_emp['name']+local_emp['business_name1'])
local_emp = local_emp.drop(['name','business_name1'],axis=1)
local_emp['emp_name'] = local_emp['emp_name'].str.upper()
local_emp['title'] = local_emp['title'].str.upper()

#sum compensation fields
local_emp.insert(6,'total_comp',local_emp.iloc[:, 3:6].sum(axis=1))
local_emp.reset_index(drop=True,inplace=True)
local_emp.rename_axis("id",inplace=True)
local_emp.to_csv("np_emp_comp_2018.csv")
local_emp.head()


,object_id,emp_name,title,org_comp,related_comp,other_cmp,total_comp,form,source,org_id
id,,,,,,,,,,
0,201902269349301505,ROY VINYARD,PRESIDENT & CEO,1236805,0,160715,1397520,IRS990,Frm990PrtVIISctnA,930223960
1,201902269349301505,MARK HETZ,CH INFO OFFICER,533924,0,204136,738060,IRS990,Frm990PrtVIISctnA,930223960
2,201902269349301505,GREG WOJTAL,CFO,470862,0,97769,568631,IRS990,Frm990PrtVIISctnA,930223960
3,201902269349301505,PAUL MACUGA,PEOPLE OFFICER,347741,0,86608,434349,IRS990,Frm990PrtVIISctnA,930223960
4,201902269349301505,DENNIE CONRAD,CHIEF STRATEGY OFFICER,434116,0,121811,555927,IRS990,Frm990PrtVIISctnA,930223960


object_id        int64
emp_name        object
title           object
org_comp         int64
related_comp    object
other_cmp        int64
total_comp       int64
form            object
source          object
ein              int64
dtype: object